In [1]:
# import necessary libraries
import numpy as np
import pandas as pd
pd.set_option('max_columns',1000000000000)
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
#read in both the test and train data 
data = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [3]:
data.head()

,Id,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37,revenue
0,0,07/17/1999,İstanbul,Big Cities,IL,4,5.0,4.0,4.0,2,2,5,4,5,5,3,5,5.0,1,2,2,2,4,5,4,1,3,3,1,1,1.0,4.0,2.0,3.0,5,3,4,5,5,4,3,4,5653753.0
1,1,02/14/2008,Ankara,Big Cities,FC,4,5.0,4.0,4.0,1,2,5,5,5,5,1,5,5.0,0,0,0,0,0,3,2,1,3,2,0,0,0.0,0.0,3.0,3.0,0,0,0,0,0,0,0,0,6923131.0
2,2,03/09/2013,Diyarbakır,Other,IL,2,4.0,2.0,5.0,2,3,5,5,5,5,2,5,5.0,0,0,0,0,0,1,1,1,1,1,0,0,0.0,0.0,1.0,3.0,0,0,0,0,0,0,0,0,2055379.0
3,3,02/02/2012,Tokat,Other,IL,6,4.5,6.0,6.0,4,4,10,8,10,10,8,10,7.5,6,4,9,3,12,20,12,6,1,10,2,2,2.5,2.5,2.5,7.5,25,12,10,6,18,12,12,6,2675511.0
4,4,05/09/2009,Gaziantep,Other,IL,3,4.0,3.0,4.0,2,2,5,5,5,5,2,5,5.0,2,1,2,1,4,2,2,1,2,1,2,3,3.0,5.0,1.0,3.0,5,1,3,2,3,4,3,3,4316715.0


In [4]:
data.set_index('Id',inplace=True)
test.set_index('Id',inplace=True)

In [5]:
data.head()

,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37,revenue
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,07/17/1999,İstanbul,Big Cities,IL,4,5.0,4.0,4.0,2,2,5,4,5,5,3,5,5.0,1,2,2,2,4,5,4,1,3,3,1,1,1.0,4.0,2.0,3.0,5,3,4,5,5,4,3,4,5653753.0
1,02/14/2008,Ankara,Big Cities,FC,4,5.0,4.0,4.0,1,2,5,5,5,5,1,5,5.0,0,0,0,0,0,3,2,1,3,2,0,0,0.0,0.0,3.0,3.0,0,0,0,0,0,0,0,0,6923131.0
2,03/09/2013,Diyarbakır,Other,IL,2,4.0,2.0,5.0,2,3,5,5,5,5,2,5,5.0,0,0,0,0,0,1,1,1,1,1,0,0,0.0,0.0,1.0,3.0,0,0,0,0,0,0,0,0,2055379.0
3,02/02/2012,Tokat,Other,IL,6,4.5,6.0,6.0,4,4,10,8,10,10,8,10,7.5,6,4,9,3,12,20,12,6,1,10,2,2,2.5,2.5,2.5,7.5,25,12,10,6,18,12,12,6,2675511.0
4,05/09/2009,Gaziantep,Other,IL,3,4.0,3.0,4.0,2,2,5,5,5,5,2,5,5.0,2,1,2,1,4,2,2,1,2,1,2,3,3.0,5.0,1.0,3.0,5,1,3,2,3,4,3,3,4316715.0


In [6]:
test.head()

,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,01/22/2011,Niğde,Other,FC,1,4.0,4.0,4.0,1,2,5,4,5,5,5,3,4.0,0,0,0,2,0,5,5,3,1,4,0,0,0.0,0.0,2.0,3.0,0,0,0,0,0,0,0,0
1,03/18/2011,Konya,Other,IL,3,4.0,4.0,4.0,2,2,5,3,4,4,2,4,5.0,0,0,0,0,0,5,5,3,2,1,0,0,0.0,0.0,1.0,3.0,0,0,0,0,0,0,0,0
2,10/30/2013,Ankara,Big Cities,FC,3,4.0,4.0,4.0,2,2,5,4,4,5,4,5,5.0,0,0,0,0,0,5,5,5,5,5,0,0,0.0,0.0,2.0,3.0,0,0,0,0,0,0,0,0
3,05/06/2013,Kocaeli,Other,IL,2,4.0,4.0,4.0,2,3,5,4,5,4,3,4,5.0,0,0,0,0,4,4,4,3,2,2,0,0,0.0,0.0,2.0,3.0,0,4,0,0,0,0,0,0
4,07/31/2013,Afyonkarahisar,Other,FC,2,4.0,4.0,4.0,1,2,5,4,5,4,3,5,4.0,0,0,0,0,0,1,5,3,1,1,0,0,0.0,0.0,5.0,3.0,0,0,0,0,0,0,0,0


In [7]:
#change the datatime column to datetime format
data['Open Date'] = data['Open Date'].astype('datetime64')
test['Open Date'] = test['Open Date'].astype('datetime64')

In [8]:
#Extract the age of each of the restaurant
today = pd.to_datetime('today')
data['age'] = (today-data['Open Date']).dt.days
test['age'] = (today-test['Open Date']).dt.days

In [9]:
data['Type'].value_counts()

FC    76
IL    60
DT     1
Name: Type, dtype: int64

In [10]:
test['Type'].value_counts()

FC    57019
IL    40447
DT     2244
MB      290
Name: Type, dtype: int64

In [11]:
#combine the two least unpopular Type
test['Type'] = test['Type'].replace({'MB':'DT'})

In [12]:
test['Type'].value_counts()

FC    57019
IL    40447
DT     2534
Name: Type, dtype: int64

In [15]:
data[data['City Group']=='Big Cities']['City'].unique()

array(['İstanbul', 'Ankara', 'İzmir'], dtype=object)

In [13]:
# Use label encoder label encoder on the City, the columns can be ranked in order importance
label_encoder = LabelEncoder()

In [14]:
data['City Group'] = label_encoder.fit_transform(data['City Group'])

In [15]:
test['City Group'] = label_encoder.transform(test['City Group'])

In [16]:
#Use onehot encoder on the type since we can put it in no particular order
one_hot_encode = OneHotEncoder(drop='first',sparse=False)

In [17]:
a = one_hot_encode.fit_transform(np.array(data['Type']).reshape(-1,1))

In [18]:
b = one_hot_encode.transform(np.array(test['Type']).reshape(-1,1))

In [19]:
data = data.join(pd.DataFrame(a,columns=list(one_hot_encode.get_feature_names(['Type']))))

In [20]:
data.head()

,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37,revenue,age,Type_FC,Type_IL
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1999-07-17,İstanbul,0,IL,4,5.0,4.0,4.0,2,2,5,4,5,5,3,5,5.0,1,2,2,2,4,5,4,1,3,3,1,1,1.0,4.0,2.0,3.0,5,3,4,5,5,4,3,4,5653753.0,7781,0.0,1.0
1,2008-02-14,Ankara,0,FC,4,5.0,4.0,4.0,1,2,5,5,5,5,1,5,5.0,0,0,0,0,0,3,2,1,3,2,0,0,0.0,0.0,3.0,3.0,0,0,0,0,0,0,0,0,6923131.0,4647,1.0,0.0
2,2013-03-09,Diyarbakır,1,IL,2,4.0,2.0,5.0,2,3,5,5,5,5,2,5,5.0,0,0,0,0,0,1,1,1,1,1,0,0,0.0,0.0,1.0,3.0,0,0,0,0,0,0,0,0,2055379.0,2797,0.0,1.0
3,2012-02-02,Tokat,1,IL,6,4.5,6.0,6.0,4,4,10,8,10,10,8,10,7.5,6,4,9,3,12,20,12,6,1,10,2,2,2.5,2.5,2.5,7.5,25,12,10,6,18,12,12,6,2675511.0,3198,0.0,1.0
4,2009-05-09,Gaziantep,1,IL,3,4.0,3.0,4.0,2,2,5,5,5,5,2,5,5.0,2,1,2,1,4,2,2,1,2,1,2,3,3.0,5.0,1.0,3.0,5,1,3,2,3,4,3,3,4316715.0,4197,0.0,1.0


In [21]:
test = test.join(pd.DataFrame(b,columns=list(one_hot_encode.get_feature_names(['Type']))))

In [22]:
test.head()

,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37,age,Type_FC,Type_IL
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2011-01-22,Niğde,1,FC,1,4.0,4.0,4.0,1,2,5,4,5,5,5,3,4.0,0,0,0,2,0,5,5,3,1,4,0,0,0.0,0.0,2.0,3.0,0,0,0,0,0,0,0,0,3574,1.0,0.0
1,2011-03-18,Konya,1,IL,3,4.0,4.0,4.0,2,2,5,3,4,4,2,4,5.0,0,0,0,0,0,5,5,3,2,1,0,0,0.0,0.0,1.0,3.0,0,0,0,0,0,0,0,0,3519,0.0,1.0
2,2013-10-30,Ankara,0,FC,3,4.0,4.0,4.0,2,2,5,4,4,5,4,5,5.0,0,0,0,0,0,5,5,5,5,5,0,0,0.0,0.0,2.0,3.0,0,0,0,0,0,0,0,0,2562,1.0,0.0
3,2013-05-06,Kocaeli,1,IL,2,4.0,4.0,4.0,2,3,5,4,5,4,3,4,5.0,0,0,0,0,4,4,4,3,2,2,0,0,0.0,0.0,2.0,3.0,0,4,0,0,0,0,0,0,2739,0.0,1.0
4,2013-07-31,Afyonkarahisar,1,FC,2,4.0,4.0,4.0,1,2,5,4,5,4,3,5,4.0,0,0,0,0,0,1,5,3,1,1,0,0,0.0,0.0,5.0,3.0,0,0,0,0,0,0,0,0,2653,1.0,0.0


In [23]:
#Extract the components of the datetime column
data['year'] = data['Open Date'].dt.year
data['month'] = data['Open Date'].dt.month
data['day'] = data['Open Date'].dt.day

In [24]:
data.head()

,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37,revenue,age,Type_FC,Type_IL,year,month,day
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1999-07-17,İstanbul,0,IL,4,5.0,4.0,4.0,2,2,5,4,5,5,3,5,5.0,1,2,2,2,4,5,4,1,3,3,1,1,1.0,4.0,2.0,3.0,5,3,4,5,5,4,3,4,5653753.0,7781,0.0,1.0,1999,7,17
1,2008-02-14,Ankara,0,FC,4,5.0,4.0,4.0,1,2,5,5,5,5,1,5,5.0,0,0,0,0,0,3,2,1,3,2,0,0,0.0,0.0,3.0,3.0,0,0,0,0,0,0,0,0,6923131.0,4647,1.0,0.0,2008,2,14
2,2013-03-09,Diyarbakır,1,IL,2,4.0,2.0,5.0,2,3,5,5,5,5,2,5,5.0,0,0,0,0,0,1,1,1,1,1,0,0,0.0,0.0,1.0,3.0,0,0,0,0,0,0,0,0,2055379.0,2797,0.0,1.0,2013,3,9
3,2012-02-02,Tokat,1,IL,6,4.5,6.0,6.0,4,4,10,8,10,10,8,10,7.5,6,4,9,3,12,20,12,6,1,10,2,2,2.5,2.5,2.5,7.5,25,12,10,6,18,12,12,6,2675511.0,3198,0.0,1.0,2012,2,2
4,2009-05-09,Gaziantep,1,IL,3,4.0,3.0,4.0,2,2,5,5,5,5,2,5,5.0,2,1,2,1,4,2,2,1,2,1,2,3,3.0,5.0,1.0,3.0,5,1,3,2,3,4,3,3,4316715.0,4197,0.0,1.0,2009,5,9


In [25]:
test['year'] = test['Open Date'].dt.year
test['month'] = test['Open Date'].dt.month
test['day'] = test['Open Date'].dt.day

In [26]:
test.head()

,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37,age,Type_FC,Type_IL,year,month,day
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2011-01-22,Niğde,1,FC,1,4.0,4.0,4.0,1,2,5,4,5,5,5,3,4.0,0,0,0,2,0,5,5,3,1,4,0,0,0.0,0.0,2.0,3.0,0,0,0,0,0,0,0,0,3574,1.0,0.0,2011,1,22
1,2011-03-18,Konya,1,IL,3,4.0,4.0,4.0,2,2,5,3,4,4,2,4,5.0,0,0,0,0,0,5,5,3,2,1,0,0,0.0,0.0,1.0,3.0,0,0,0,0,0,0,0,0,3519,0.0,1.0,2011,3,18
2,2013-10-30,Ankara,0,FC,3,4.0,4.0,4.0,2,2,5,4,4,5,4,5,5.0,0,0,0,0,0,5,5,5,5,5,0,0,0.0,0.0,2.0,3.0,0,0,0,0,0,0,0,0,2562,1.0,0.0,2013,10,30
3,2013-05-06,Kocaeli,1,IL,2,4.0,4.0,4.0,2,3,5,4,5,4,3,4,5.0,0,0,0,0,4,4,4,3,2,2,0,0,0.0,0.0,2.0,3.0,0,4,0,0,0,0,0,0,2739,0.0,1.0,2013,5,6
4,2013-07-31,Afyonkarahisar,1,FC,2,4.0,4.0,4.0,1,2,5,4,5,4,3,5,4.0,0,0,0,0,0,1,5,3,1,1,0,0,0.0,0.0,5.0,3.0,0,0,0,0,0,0,0,0,2653,1.0,0.0,2013,7,31


In [ ]:
city_mapper = {
    ['']:'Istanbul'
}

In [29]:
#drop the unneccesary columns
data.drop(['Open Date','City','Type'],1,inplace=True)

In [30]:
data.head()

,City Group,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37,revenue,age,Type_FC,Type_IL,year,month,day
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,4,5.0,4.0,4.0,2,2,5,4,5,5,3,5,5.0,1,2,2,2,4,5,4,1,3,3,1,1,1.0,4.0,2.0,3.0,5,3,4,5,5,4,3,4,5653753.0,7781,0.0,1.0,1999,7,17
1,0,4,5.0,4.0,4.0,1,2,5,5,5,5,1,5,5.0,0,0,0,0,0,3,2,1,3,2,0,0,0.0,0.0,3.0,3.0,0,0,0,0,0,0,0,0,6923131.0,4647,1.0,0.0,2008,2,14
2,1,2,4.0,2.0,5.0,2,3,5,5,5,5,2,5,5.0,0,0,0,0,0,1,1,1,1,1,0,0,0.0,0.0,1.0,3.0,0,0,0,0,0,0,0,0,2055379.0,2797,0.0,1.0,2013,3,9
3,1,6,4.5,6.0,6.0,4,4,10,8,10,10,8,10,7.5,6,4,9,3,12,20,12,6,1,10,2,2,2.5,2.5,2.5,7.5,25,12,10,6,18,12,12,6,2675511.0,3198,0.0,1.0,2012,2,2
4,1,3,4.0,3.0,4.0,2,2,5,5,5,5,2,5,5.0,2,1,2,1,4,2,2,1,2,1,2,3,3.0,5.0,1.0,3.0,5,1,3,2,3,4,3,3,4316715.0,4197,0.0,1.0,2009,5,9


In [ ]:
for i in data['City Group']

In [31]:
test.drop(['Open Date','City','Type'],1,inplace=True)

In [32]:
test.head()

,City Group,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37,age,Type_FC,Type_IL,year,month,day
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1,1,4.0,4.0,4.0,1,2,5,4,5,5,5,3,4.0,0,0,0,2,0,5,5,3,1,4,0,0,0.0,0.0,2.0,3.0,0,0,0,0,0,0,0,0,3574,1.0,0.0,2011,1,22
1,1,3,4.0,4.0,4.0,2,2,5,3,4,4,2,4,5.0,0,0,0,0,0,5,5,3,2,1,0,0,0.0,0.0,1.0,3.0,0,0,0,0,0,0,0,0,3519,0.0,1.0,2011,3,18
2,0,3,4.0,4.0,4.0,2,2,5,4,4,5,4,5,5.0,0,0,0,0,0,5,5,5,5,5,0,0,0.0,0.0,2.0,3.0,0,0,0,0,0,0,0,0,2562,1.0,0.0,2013,10,30
3,1,2,4.0,4.0,4.0,2,3,5,4,5,4,3,4,5.0,0,0,0,0,4,4,4,3,2,2,0,0,0.0,0.0,2.0,3.0,0,4,0,0,0,0,0,0,2739,0.0,1.0,2013,5,6
4,1,2,4.0,4.0,4.0,1,2,5,4,5,4,3,5,4.0,0,0,0,0,0,1,5,3,1,1,0,0,0.0,0.0,5.0,3.0,0,0,0,0,0,0,0,0,2653,1.0,0.0,2013,7,31


In [33]:
data.shape, test.shape

((137, 45), (100000, 44))

In [35]:
#split the dataframe into features and target
X = data.drop('revenue',1)
y = data['revenue']

In [36]:
#instanciate the crossvalidation fold and the params, which would be tuned manually
kfold = KFold()

params = {'n_estimators':130,
         'learning_rate':0.04,
          'max_depth':1,
          'subsample':0.9,
          'loss':'huber',
          'criterion':'mae',
         'random_state':500}

In [37]:
score = 0 # define a value to be updated, it gives us a general overlook of how well our model is doing as we tune manually
gbm_results = pd.DataFrame()# define a dataframe that will contain our test prediction after training on each fold
#iterate through each fold
for i,(train_index,val_index) in enumerate(kfold.split(X,y)):
    #define test and train splits
    X_train,y_train = X.iloc[train_index],y.iloc[train_index]
    X_test,y_test = X.iloc[val_index],y.iloc[val_index]
    #fit model, predict and score
    model = GradientBoostingRegressor(**params)
    print(f'************ fold {i}**************')
    model.fit(X_train,y_train)
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test,preds))
    print(rmse)
    score += rmse/5 #updating the average
    result = model.predict(test)
    
    gbm_results['Fold'+str(i)] = result
    
print(f'Average score is {score}')

************ fold 0**************
3410907.6534201694
************ fold 1**************
1611860.9090101263
************ fold 2**************
2667396.8169864
************ fold 3**************
2456923.5159307322
************ fold 4**************
2060427.994638586
Average score is 2441503.377997203


In [38]:
gbm_results

,Fold0,Fold1,Fold2,Fold3,Fold4
0,4.649671e+06,4.584940e+06,4.303003e+06,4.399092e+06,4.361424e+06
1,3.340150e+06,2.844762e+06,3.279458e+06,3.886666e+06,3.422370e+06
2,3.452343e+06,3.070383e+06,4.537676e+06,2.666912e+06,3.167638e+06
3,3.574592e+06,3.001687e+06,3.373533e+06,2.637900e+06,3.099233e+06
4,3.567754e+06,2.975294e+06,4.240488e+06,2.747253e+06,3.120270e+06
...,...,...,...,...,...
99995,4.565124e+06,5.633923e+06,4.808323e+06,4.520963e+06,4.783987e+06
99996,3.159207e+06,3.381715e+06,3.415230e+06,3.951696e+06,3.479878e+06
99997,3.289812e+06,3.067671e+06,4.106682e+06,3.210401e+06,3.176355e+06
99998,3.575243e+06,3.244212e+06,3.787629e+06,3.103970e+06,3.433931e+06


In [39]:
kfold = KFold(n_splits=7)

params = {'max_features':0.09,
          'n_estimators':125,
          'max_depth':6,
          'min_samples_leaf':5,
         'random_state':500}

In [40]:
score = 0
rf_results = pd.DataFrame()
for i,(train_index,val_index) in enumerate(kfold.split(X,y)):
    X_train,y_train = X.iloc[train_index],y.iloc[train_index]
    X_test,y_test = X.iloc[val_index],y.iloc[val_index]
    
    model = RandomForestRegressor(**params)
    print(f'************ fold {i}**************')
    model.fit(X_train,y_train)
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test,preds))
    print(rmse)
    score += rmse/5
    result = model.predict(test)
    
    rf_results['Fold'+str(i)] = result
    
print(f'Average score is {score}')

************ fold 0**************
3652252.120475039
************ fold 1**************
1783115.8828273485
************ fold 2**************
1789017.900619643
************ fold 3**************
2976594.8518392807
************ fold 4**************
1332701.186495728
************ fold 5**************
2745167.378522557
************ fold 6**************
1944325.7225961874
Average score is 3244635.0086751566


In [41]:
rf_results

,Fold0,Fold1,Fold2,Fold3,Fold4,Fold5,Fold6
0,4.472031e+06,4.722187e+06,4.453718e+06,4.585228e+06,4.877682e+06,4.640552e+06,4.574542e+06
1,3.893224e+06,3.827603e+06,3.697193e+06,3.939144e+06,3.976658e+06,3.897279e+06,3.928442e+06
2,4.303873e+06,4.381766e+06,4.238926e+06,4.236599e+06,4.444641e+06,4.202286e+06,4.307034e+06
3,3.866550e+06,3.912122e+06,3.561506e+06,3.836824e+06,3.645750e+06,3.665138e+06,3.663222e+06
4,4.207452e+06,4.392933e+06,4.131026e+06,4.330753e+06,4.418055e+06,4.134543e+06,4.372123e+06
...,...,...,...,...,...,...,...
99995,4.626220e+06,5.687256e+06,5.397282e+06,5.060629e+06,4.912439e+06,5.135801e+06,5.008021e+06
99996,3.261971e+06,3.286411e+06,3.277663e+06,3.354082e+06,3.447736e+06,3.276140e+06,3.247503e+06
99997,4.642071e+06,4.826814e+06,4.466177e+06,4.540278e+06,4.805460e+06,4.632092e+06,5.055108e+06
99998,4.541059e+06,4.556801e+06,4.187218e+06,4.618693e+06,4.655933e+06,4.511177e+06,4.497968e+06


In [42]:
sub = pd.read_csv('../data/sampleSubmission.csv')

In [43]:
#blend the predictions
sub['Prediction'] = np.mean(gbm_results,1)*0.58+ np.mean(rf_results,1)*0.42

In [44]:
sub.to_csv('../data/blendsub.csv',index=False)